In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib as mpl
import matplotlib.pyplot as plt 
import pymssql
# import seaborn as sns
import sklearn
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from sklearn.metrics import mean_absolute_error
import seaborn as sns
from keras.preprocessing.sequence import TimeseriesGenerator

2022-11-29 12:49:39.524549: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-29 12:49:39.524605: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# def get_data():
#     df=pd.DataFrame()
#     try:
#         con = pymssql.connect(user='ds_rcm',password = 'mtbc@123'
#                      ,host='172.16.0.32',database='mis_db',autocommit = True)
#     except Exception as e:
#         print('unable to make connection',e)
#     cur=con.cursor()

#     query = """
    
#         IF OBJECT_ID('TEMPDB..#CLAIMS') IS NOT NULL
#         DROP TABLE #CLAIMS
#         SELECT CONVERT(DATE,C.DOS) AS DOS,CLAIM_NO INTO #CLAIMS FROM CLAIMS_SUBMITTED AS C
#         JOIN PATIENT AS P ON P.PATIENT_ACCOUNT=C.PATIENT_ACCOUNT
#         JOIN PRACTICES AS PR ON PR.PRACTICE_CODE=P.PRACTICE_CODE
#         AND PR.IS_ACTIVE=1 AND ISNULL(PR.IS_TEST_PRACTICE,0)=0
#         AND ISNULL(PR.EXCLUDE_FROM_BILLING_REPORTS,0) = 0
#         AND PR.EMR_NAME NOT IN ('CHARTS PRO','PM STANDALONE','TESTINGPRACTICE')
#         AND PR.PRACTICE_CODE NOT LIKE '9090%' AND PR.PRAC_NAME NOT LIKE '%TEST%'
#         WHERE ISNULL(C.DELETED,0)=0 AND CONVERT(DATE,C.DOS) BETWEEN '01-01-2021' AND '10-20-2022'----6383824



# #         ---------------------------#CHARGES----------------------
# #         IF OBJECT_ID('TEMPDB..#CHARGES') IS NOT NULL
# #         DROP TABLE #CHARGES
# #         SELECT C.Claim_No,CC.claim_charges_id,CC.Amount INTO #CHARGES FROM Claim_Charges AS CC
# #         JOIN #CLAIMS AS C ON CC.Claim_No=C.Claim_No
# #         WHERE ISNULL(CC.DELETED,0)=0------



# #         ---------------------------------------------------------
# #         SELECT CONVERT(DATE,C.DOS)AS DOS,COUNT(DISTINCT C.Claim_No) AS CLAIM_COUNT,SUM(CC.Amount) AS CHARGES_SUM FROM #CLAIMS AS C LEFT JOIN #CHARGES AS CC
# #         ON C.Claim_No=CC.Claim_No
# #         GROUP BY CONVERT(DATE,C.DOS)
# #         ORDER BY CONVERT(DATE,C.DOS) ASC;
#         """
#     try:
#         cur.execute(query)
#         df=pd.DataFrame(cur.fetchall(),columns=[x[0] for x in cur.description])
# #         print(df)
# #         df.columns=df.columns.str.upper()
#         con.close()
#     except:
#         print('Error in reading data.....')
#         con.close()
#     return df

In [3]:
def get_data():
    df=pd.DataFrame()
    try:
        con = pymssql.connect(user='ds_rcm',password = 'mtbc@123'
                     ,host='172.16.0.32',database='mis_db',autocommit = True)
    except Exception as e:
        print('unable to make connection',e)
    cur=con.cursor()

    query = """
    
SELECT CONVERT(DATE,CS.Process_Date) AS PROCESS_DATE,COUNT(DISTINCT CS.CLAIM_NO) CLAIM_COUNT
FROM CLAIMS_SUBMITTED AS CS
JOIN CLAIMS AS C ON C.Claim_No=CS.Claim_No
JOIN PATIENT AS P ON P.PATIENT_ACCOUNT=C.PATIENT_ACCOUNT
JOIN PRACTICES AS PR ON PR.PRACTICE_CODE=P.PRACTICE_CODE
AND PR.IS_ACTIVE=1 AND ISNULL(PR.IS_TEST_PRACTICE,0)=0
AND ISNULL(PR.EXCLUDE_FROM_BILLING_REPORTS,0) = 0
AND PR.EMR_NAME NOT IN ('CHARTS PRO','PM STANDALONE','TESTINGPRACTICE')
AND PR.PRACTICE_CODE NOT LIKE '9090%' AND PR.PRAC_NAME NOT LIKE '%TEST%'
WHERE ISNULL(CS.DELETED,0)=0 AND CONVERT(DATE,CS.Process_Date) BETWEEN '01-01-2020' AND '06-20-2022'
GROUP BY CONVERT(DATE,CS.Process_Date)
ORDER BY CONVERT(DATE,CS.Process_Date) ASC
        """
    try:
        cur.execute(query)
        df=pd.DataFrame(cur.fetchall(),columns=[x[0] for x in cur.description])
#         print(df)
#         df.columns=df.columns.str.upper()
        con.close()
    except:
        print('Error in reading data.....')
        con.close()
    return df

In [5]:
claims = get_data()

In [ ]:
# df.head()

In [ ]:
claims.to_csv('Submission_Claims_Data.csv')

In [ ]:
claims = pd.read_csv('Submission_Claims_Data.csv')

In [ ]:
claims.head()

In [ ]:
len(claims)

In [ ]:
mean_claims = claims['CLAIM_COUNT'].mean()

In [ ]:
mean_claims

In [ ]:
# df.tail(30)

In [ ]:
claims.isnull().sum()

In [ ]:
claims['CLAIM_COUNT'].mask( claims['CLAIM_COUNT'] >= 40000, 40000 , inplace=True )

In [ ]:
# new_claims = claims.loc[ claims['CLAIM_COUNT'] >= 50000, 'CLAIM_COUNT'] = 50000

In [ ]:
# new_claims

In [ ]:
claims = claims[["PROCESS_DATE", "CLAIM_COUNT"]]

In [ ]:
claims["PROCESS_DATE"] = pd.to_datetime(claims["PROCESS_DATE"])
claims = claims.sort_values('PROCESS_DATE')
claims =claims.set_index("PROCESS_DATE")

In [ ]:
claims.head()

In [ ]:
# claims.loc[:]

In [ ]:
claims.plot(figsize=(20,12))

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
dec= seasonal_decompose(claims['CLAIM_COUNT'])

In [ ]:
dec.plot();

In [ ]:
train = claims.loc[:'2022-04-30']
test = claims.loc['2022-05-01':]

In [ ]:
len(train)

In [ ]:
len(test)

In [ ]:
scaler = MinMaxScaler()

In [ ]:
scaler.fit(train)

In [ ]:
scaled_train = scaler.transform(train)

In [ ]:
scaled_test = scaler.transform(test) 

In [ ]:
len(scaled_train)

In [ ]:
# define generator
n_input = 32
n_features = 1
generator = TimeseriesGenerator(scaled_train, scaled_train, length=n_input, batch_size=16
                               )

In [ ]:
model = Sequential()

model.add(LSTM(units =128, activation='relu', return_sequences=True, input_shape = (n_input, n_features)))
model.add(Dropout(0.2))
model.add(LSTM(units =128, activation='relu', return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units =128, activation='relu', return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=1)) # Prediction of the next value
model.compile(optimizer='adam', loss='mse')

In [ ]:
model.summary()

In [ ]:
# fit model
model.fit(generator,epochs=250)

In [ ]:
loss_per_epoch = model.history.history['loss']
plt.plot(range(len(loss_per_epoch)),loss_per_epoch)

In [ ]:
first_eval_batch = scaled_train[-32:]

In [ ]:
# first_eval_batch

In [ ]:
first_eval_batch = first_eval_batch.reshape((1,n_input,n_features))

In [ ]:
model.predict(first_eval_batch)

In [ ]:
#holding my predictions
test_predictions = []


# last n_input points from the training set
first_eval_batch = scaled_train[-n_input:]
# reshape this to the format RNN wants (same format as TimeseriesGeneration)
current_batch = first_eval_batch.reshape((1,n_input,n_features))

#how far into the future will I forecast?

for i in range(len(test)):
    
    # One timestep ahead of historical 12 points
    current_pred = model.predict(current_batch)[0]
    
    #store that prediction
    test_predictions.append(current_pred)
    
    # UPDATE current batch o include prediction
    current_batch = np.append(current_batch[:,1:,:],[[current_pred]], axis= 1)

In [ ]:
# test_predictions

In [ ]:
true_predictions = scaler.inverse_transform(test_predictions)

In [ ]:
# true_predictions

In [ ]:
test['Predictions'] =true_predictions

In [ ]:
test.head(20)

In [ ]:
# accu = 

In [ ]:
test.plot(figsize=(12,8))

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt
rmse=sqrt(mean_squared_error(test['CLAIM_COUNT'],test['Predictions']))
print(rmse)

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    """Calculates MAPE given y_true and y_pred"""
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
mean_absolute_percentage_error(test['CLAIM_COUNT'],test['Predictions'])